In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging

from jax import random
import wandb

from src.models import make_Hard_OvR_Ens_loss as make_loss
from src.data import get_image_dataset, NumpyLoader
from src.utils.training import setup_training, train_loop
from experiments.configs.mnist_hard_ovr_classification import get_config

In [3]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'train_hard_ivr_classifier.ipynb'
# ^ W&B doesn't know how to handle VS Code notebooks.

wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find train_hard_ivr_classifier.ipynb.
wandb: Currently logged in as: jamesallingham (invariance-learners). Use `wandb login --relogin` to force relogin


True

In [4]:
rng = random.PRNGKey(0)

In [5]:
config = get_config()

In [6]:
train_dataset, test_dataset, val_dataset = get_image_dataset(
    dataset_name=config.dataset_name,
    val_percent=config.val_percent,
    flatten_img=True,
)
train_loader = NumpyLoader(train_dataset, config.batch_size)
val_loader = NumpyLoader(val_dataset, config.batch_size)
test_loader = NumpyLoader(test_dataset, config.batch_size)

In [7]:
setup_rng, rng = random.split(rng)
init_x = train_dataset[0][0]
init_y = train_dataset[0][1]

model, state = setup_training(config, setup_rng, init_x, init_y)

+---------------------------------------------+------------+--------+-----------+--------+
| Name                                        | Shape      | Size   | Mean      | Std    |
+---------------------------------------------+------------+--------+-----------+--------+
| batch_stats/nets_0/layer_0/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_0/layer_0/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |
| batch_stats/nets_1/layer_0/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_1/layer_0/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |
| batch_stats/nets_1/layer_1/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_1/layer_1/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |

In [8]:
state = train_loop(
    model, state, config, rng, make_loss, make_loss, train_loader, val_loader,
    # test_loader,
    wandb_kwargs={
        'mode': 'offline',
        # 'notes': '',
    },
)

  0%|          | 0/50 [00:00<?, ?it/s]

ys = DeviceArray([6], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([8], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([0], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([5], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([4], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([5], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([6], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([2], dtype=int32)
prod_ll = DeviceArray(nan, dtype=float32)
logZ = DeviceArray(nan, dtype=float32)
ys = DeviceArray([6], dtype=int32)
prod_ll = DeviceArray(nan, dtype=floa

KeyboardInterrupt: 